In [21]:
import numpy as np
import os
import pickle

In [2]:
import progressbar
from bert_serving.client import BertClient
bc = BertClient(ip='10.15.82.136')

In [3]:
root_folder = '/home/lyt/code/vqa-concept'
data_folder = '{}/dataTVQA'.format(root_folder)
fea_folder = '{}/image-feature/bottomup'.format(data_folder)

In [5]:
with open('{}/objects_vocab.txt'.format(data_folder)) as f:
    objects_vocab = f.read().splitlines()
objects_vocab = ['__no_objects__'] + objects_vocab

In [4]:
emb_size = 768

In [15]:
obj_emb = np.zeros((len(objects_vocab), emb_size), dtype='float32')
fill_cnt = 0
for i, line in enumerate(objects_vocab):
    synonyms = line.split(',')
    curr_embs = bc.encode(synonyms)
    obj_emb[i] = np.mean(curr_embs, axis=0)
print(obj_emb.shape)

(1601, 768)


In [22]:
pcafile = '{}/ocr_bert.new.{}/trval/pca.0init.pkl'.format(fea_folder, 300)
assert(os.path.exists(pcafile))
with open(pcafile, 'rb') as f:
    pca=pickle.load(f)
obj_emb = pca.transform(obj_emb)
print(obj_emb.shape)

/home/lyt/code/vqa-concept/dataTVQA/image-feature/bottomup/ocr_bert.new.300/trval/pca.0init.pkl
(1601, 300)


In [24]:
trainfile = '{}/{}_100_class-prob.npy'.format(fea_folder, 'train')
trainX = np.load(trainfile)
print(trainX.shape)

(21953, 100, 1601)


In [25]:
trX = np.dot(trainX,obj_emb)
print(trX.shape)

(21953, 100, 300)


In [26]:
trainfile = '{}/{}_100_class-emb.new.npy'.format(fea_folder, 'train')
np.save(trainfile,trX)
trainX = None
trX = None

In [27]:
valfile = '{}/{}_100_class-prob.npy'.format(fea_folder, 'val')
valX = np.load(valfile)
print(valX.shape)

(3166, 100, 1601)


In [28]:
vaX = np.dot(valX,obj_emb)
print(vaX.shape)

(3166, 100, 300)


In [29]:
valfile = '{}/{}_100_class-emb.new.npy'.format(fea_folder, 'val')
np.save(valfile,vaX)
valX = None
vaX = None

In [30]:
testfile = '{}/{}_100_class-prob.npy'.format(fea_folder, 'test')
testX = np.load(testfile)
print(testX.shape)

(3289, 100, 1601)


In [31]:
teX = np.dot(testX,obj_emb)
print(teX.shape)

(3289, 100, 300)


In [32]:
testfile = '{}/{}_100_class-emb.new.npy'.format(fea_folder, 'test')
np.save(testfile,teX)